# Practical task 
You will model the famous [Keeling curve](https://www.esrl.noaa.gov/gmd/ccgg/trends/) using contemporary deep learning frameworks. This notebook will download the data and read it in as numpy arrays and provide example code in [pytorch](https://pytorch.org/). You are welcome to use other frameworks and change the dataset to better fit your model arhitecture. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os 
%matplotlib inline
%matplotlib notebook


You can download the dataset from: ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt

or run the cell below which will check if the dataset is present in working directory and download it there if not.

In [ ]:
url="ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt"
filename=os.path.split(url)[-1]

if not os.path.exists(filename):    
    from urllib import request
    print('Downloading data from:\n'+url)
    request.urlretrieve(url,filename)


Read in the data as `numpy` array

In [ ]:
data=np.genfromtxt(filename,comments='#',missing_values='-99.99',usecols=[0,1,3])

Xfull=data[:,0]+(data[:,1]-1)/12.0
Yfull=data[:,-1]

missing_values=Yfull==-99.99

Xfull=Xfull[~missing_values].reshape(-1,1)
Yfull=Yfull[~missing_values].reshape(-1,1)

data=[]


# Use last 3 years as a test set 
test_length=3*12

Xtrain_np = Xfull[:-test_length]
Ytrain_np = Yfull[:-test_length]

Xtest_np = Xfull[-test_length:]
Ytest_np = Yfull[-test_length:]

# Data for prediction
Xpred_np = Xtrain_np[-1]+np.arange(1,40*12).reshape(-1,1)/12


Plot the train and test data

In [ ]:
plt.scatter(Xtrain_np,Ytrain_np,marker='.',linewidths=0.05)
plt.scatter(Xtest_np,Ytest_np,marker='.',linewidths=0.05)
plt.title(r'$CO_2$ concentration')
plt.xlabel('year')
plt.ylabel(r'$CO_2$ [ppm]')
plt.show()

# Preprocess the data
You are allowed to change the preprocessing and structure of the dataset to better suit your model architecture.

Below is a simple example.

In [ ]:
xshift=np.mean(Xtrain_np)
yshift=np.mean(Ytrain_np)

xscale=np.std(Xtrain_np)
yscale=np.std(Ytrain_np)


Xtrain_np-=xshift
Xtrain_np/=xscale
Ytrain_np-=yshift
Ytrain_np/=yscale

Xtest_np-=xshift
Xtest_np/=xscale
Ytest_np-=yshift
Ytest_np/=yscale


Xpred_np-=xshift
Xpred_np/=xscale

# Task:
Use a deep learning framwework of your choice to build and train a model to predict the CO$_2$ concentration up until year 2060.
Below is a Pytorch example with 2 ways to construct a model that you can use as a starting point. 

See the [nn module](https://pytorch.org/docs/stable/nn.html#) of Pytorch for inspiration on models and modules.

In [ ]:
import torch

if torch.cuda.is_available():  
    device = torch.device("cuda:0") 
else:  
    device = torch.device("cpu")
print(device)
# Load the data into pytorch tensors for training
Xtrain=torch.FloatTensor(Xtrain_np).to(device)
Ytrain=torch.FloatTensor(Ytrain_np).to(device)

Xtest=torch.FloatTensor(Xtest_np).to(device)
Ytest=torch.FloatTensor(Ytest_np).to(device)


In [ ]:
import torch.nn as nn



#############
# 1st way to define model
#############
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.linear1=nn.Linear(1,100)
        self.linear2=nn.Linear(100,1)
    
    def forward(self,x):
        x=nn.ReLU(self.linear1(x))
        x=self.linear2(x)
        return x
                   
model = NN


#############
# 2nd way to define model
#############
model = torch.nn.Sequential(
    nn.Linear(1,100),
    nn.ReLU(),
    nn.Linear(100,1)
)


model.to(device)

criterion=nn.MSELoss()

# Change optimizer and hyperparameters
optimizer=torch.optim.SGD(model.parameters(),lr=1e-3,momentum=0.9,weight_decay=0)


NUM_EPOCHS=500

for i in range(NUM_EPOCHS):
    optimizer.zero_grad()
    pred=model(Xtrain)
    loss=criterion(pred,Ytrain)
    loss.backward()
    optimizer.step()
    if i%100==0:
        with torch.no_grad():
            test_loss=criterion(model(Xtest),Ytest)
        print(f'{i}: {loss.item()}   {test_loss.item()}')




Plot the prediction and extrapolation for analysis

In [ ]:
Xpred=torch.FloatTensor(Xpred_np).to(device)

with torch.no_grad():
    pred=model(Xtrain)
    predf=model(Xpred)
    
plt.plot(Xtrain.cpu().numpy()*xscale+xshift,pred.cpu().numpy()*yscale+yshift,color='C0',label='Prediction')
plt.plot(Xtrain_np*xscale+xshift,Ytrain_np*yscale+yshift,'x',color='C0',alpha=0.2,label='Train data')
plt.plot(Xtest_np*xscale+xshift,Ytest_np*yscale+yshift,'o',color='C1',alpha=0.2,label='Test data')
plt.plot(Xpred.cpu().numpy()*xscale+xshift,predf.cpu().numpy()*yscale+yshift,color='C3',label='Extrapolation')

plt.legend()
plt.xlabel(r'Year')
plt.ylabel(r'CO$_2$ [ppm]')
plt.show()

# Question:
**How is your model performing, is the behaviour expected?** 

*answer:*


**Do you trust the extrapolation why/why not?** 

*answer:*

**How would you change the architecture or dataset to improve the extrapolation?**

*answer:*
